# Задача 1. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [72]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

#сортируем пользоватей, которые сделали более 100 оценок 
ratings_grouped = ratings.groupby('userId').count()['rating'].sort_values(ascending=False).reset_index()
ratings_filtered = ratings_grouped[ratings_grouped.rating >= 100]
film_fans_user_ids = ratings_filtered['userId'].tolist()
user_over100 = ratings[ ratings['userId'].isin(film_fans_user_ids) ]

#создаём сводную таблицу по пользователям с о одновременно ищем для них минимум и максимум 'timestamp'
answer = user_over100.pivot_table(index=['userId'], values=['timestamp'], aggfunc= [max, min]).reset_index()

#Вычисляем разницу между максимальным и минимальным 'timestamp'
answer['average'] = answer['max'] - answer['min']

#вывод 10 строк итогового DataFrame
answer.head(10)



,userId,max,min,average
,,timestamp,timestamp,
0,4,949982274,949778714,203560
1,5,1163375145,1163373044,2101
2,8,1154474527,1154389340,85187
3,15,1469330735,997937239,471393496
4,17,1127476640,1127468587,8053
5,19,855195373,855190091,5282
6,21,854522908,853157476,1365432
7,22,1131753381,1131661890,91491
8,23,1166728253,1148386124,18342129


# Задача 2. 
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [75]:
import pandas as pd

#Определение DF - условия задачи

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

#таблица с тремя типами выручки для каждого client_id без указания адреса клиента
    #table_serv - промежуточный датафрейм куда последовательно добавляются данные
    
df_serv = pd.merge(client_base['client_id'], rzd, on='client_id', how='left')
df_serv = pd.merge(df_serv, auto, on='client_id', how='left')
answer_1 = pd.merge(df_serv, air, on='client_id', how='left').fillna(0)
answer_1.head(10)
#answer_1.head(10)

#таблицa по типам выручки с указанием адреса клиента
table_serv = pd.merge(client_base, rzd, on='client_id', how='left')
table_serv = pd.merge(table_serv, auto, on='client_id', how='left')
answer_2 = pd.merge(table_serv, air, on='client_id', how='left').fillna(0)
print(answer_2.head(10))
#answer_2.head(10)

answer_1.head(10), answer_2.head(10)

   client_id          address  rzd_revenue  auto_revenue  air_revenue
0        111  Комсомольская 4       1093.0           0.0          0.0
1        112   Энтузиастов 8а       2810.0           0.0          0.0
2        113  Левобережная 1а      10283.0       57483.0          0.0
3        114          Мира 14       5774.0          83.0          0.0
4        115        ЗЖБИиДК 1        981.0         912.0         81.0
5        116    Строителей 18          0.0        4834.0          4.0
6        117  Панфиловская 33          0.0          98.0         13.0
7        118     Мастеркова 4          0.0           0.0        173.0


(   client_id  rzd_revenue  auto_revenue  air_revenue
 0        111       1093.0           0.0          0.0
 1        112       2810.0           0.0          0.0
 2        113      10283.0       57483.0          0.0
 3        114       5774.0          83.0          0.0
 4        115        981.0         912.0         81.0
 5        116          0.0        4834.0          4.0
 6        117          0.0          98.0         13.0
 7        118          0.0           0.0        173.0,
    client_id          address  rzd_revenue  auto_revenue  air_revenue
 0        111  Комсомольская 4       1093.0           0.0          0.0
 1        112   Энтузиастов 8а       2810.0           0.0          0.0
 2        113  Левобережная 1а      10283.0       57483.0          0.0
 3        114          Мира 14       5774.0          83.0          0.0
 4        115        ЗЖБИиДК 1        981.0         912.0         81.0
 5        116    Строителей 18          0.0        4834.0          4.0
 6        117  П

# Задача 3. 
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Поскольку запуск приожения может означать визит в магазин. Добавил бы еще 2 столбца в таблицу визитов: latitude (широта) и longitude (широта). Далее, для каждой покупки по user_id и дате сопоставил из таблице визитов широту и долготу.